In [1]:
import cv2
import numpy as np
import matplotlib
matplotlib.use("TkAgg") # have to use this for tkinter to  work below
from matplotlib import pyplot as plt
%matplotlib tk

# scikit image
import skimage
from skimage import io

import os
import pandas as pd
import time

/Users/callinswitzer/anaconda/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [14]:
%qtconsole

In [3]:
import Tkinter as tk
import tkFileDialog

def file_choose():
    '''
    Opens tkfileDialog and returns full path name.
    It's the equivalent of file.choose in R.
    Make sure the following modeules are loaded
    import Tkinter as tk
    import tkFileDialog
    Also, won't work if %matplotlib qt4 has been used
    '''
    root = tk.Tk()
    root.update()
    file1 = tkFileDialog.askopenfilename(parent=root)
    root.destroy() # closes the window
    return file1

In [4]:
# adjust gamma
def adjust_gamma(image, gamma=1.0):
    # build a lookup table mapping the pixel values [0, 255] to
    # their adjusted gamma values
    invGamma = 1.0 / gamma
    table = np.array([((i / 255.0) ** invGamma) * 255
        for i in np.arange(0, 256)]).astype("uint8")
 
    # apply gamma correction using the lookup table
    return cv2.LUT(image, table)

In [5]:
# choose video
# 2016-06-16_000011.avi
aa = file_choose()

In [6]:
print aa
cap = cv2.VideoCapture(aa)
print 'numFrames = ' + str(cap.get(cv2.cv.CV_CAP_PROP_FRAME_COUNT))

/Volumes/GMD/June11MtNLaurelsAtCFS/20150611_103818.avi
numFrames = 812.0


In [7]:
# change directory
newD = '/Users/callinswitzer/Dropbox/ExperSummer2016/Kalmia/AnnotatedVideoFrom2015'
if not os.path.isdir(newD):
    os.mkdir(newD)
    
os.chdir(newD)

In [8]:
# get digitized points

#ptsFile = '/Users/callinswitzer/Dropbox/ExperSummer2016/Kalmia/DigitizedPollen2/DLTdv5_data_xypts.csv'
ptsFile = file_choose()

digPts = pd.read_csv(ptsFile)

# show frames with points digitized
digPts.dropna(thresh = 1)


,pt1_cam1_X,pt1_cam1_Y,pt2_cam1_X,pt2_cam1_Y,pt3_cam1_X,pt3_cam1_Y,pt4_cam1_X,pt4_cam1_Y
0,291.46113,103.746212,353.364868,97.483246,226.209143,22.870520,226.209143,22.870520
1,NaN,NaN,NaN,NaN,226.209143,22.870520,226.439865,22.652015
2,NaN,NaN,NaN,NaN,227.670010,23.752842,227.868774,23.439373
3,NaN,NaN,NaN,NaN,229.482978,24.856388,229.705942,24.664152
4,NaN,NaN,NaN,NaN,231.768894,26.748180,232.097177,26.588805
5,NaN,NaN,NaN,NaN,233.409097,28.725830,233.642732,28.542619
6,NaN,NaN,NaN,NaN,234.670292,30.932922,234.878666,30.778051
7,NaN,NaN,NaN,NaN,236.527052,33.700545,236.752676,33.604427
8,NaN,NaN,NaN,NaN,237.613081,36.433134,238.249704,36.175132
9,NaN,NaN,NaN,NaN,238.418845,39.165724,239.499592,38.813859


In [22]:
# plot points on each frame of video

cap = cv2.VideoCapture(aa)
ctr = 0

# reset frame num to zero
cap.set(cv2.cv.CV_CAP_PROP_FRAME_COUNT, 0) 


while(ctr <= 50):  # check !
    # capture frame-by-frame
    ret, frame = cap.read()

    if ret and ctr > 0: # if we can return a frame, and it's over a certain frame number
        # our operation on frame come here
        trueCol = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        # adjust gamma
        trueCol = adjust_gamma(trueCol, 1)
        
        plt.clf()

        # Display the resulting frame
        io.imshow(trueCol)
        
        # add points
        plt.axis([0, frame.shape[1], frame.shape[0], 0])
        plt.axis('off')
        
        plt.gca().set_position([0,0,1,1])
        plt.gca().set_aspect('equal')
        
#         plt.scatter(x = digPts.ix[ctr][[4,6]], 
#             y = trueCol.shape[0]-digPts.ix[ctr][[5,7]], 
#            s = 100, 
#            alpha = 0.5, edgecolor =  '', 
#            color = ['#deebf7', '#9ecae1', '#3182bd'])
        
        # save figure
        plt.savefig(str(ctr).zfill(4) + '.png')
        
        
    print ctr
    ctr +=1

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
# When everything is done release the capture
cap.release()
cv2.destroyAllWindows()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50


In [23]:
# use ffmpeg to convert directory to video
stta = time.time()
!ffmpeg -start_number 0 -i %04d.png -vf "scale=trunc(iw/2)*2:trunc(ih/2)*2" -c:v libx264 -pix_fmt yuv420p -y pollenAnnotatedVid3_2015_noPts.mp4
print time.time() - stta # ~ 1.4 seconds

ffmpeg version 3.1.4 Copyright (c) 2000-2016 the FFmpeg developers
  built with Apple LLVM version 8.0.0 (clang-800.0.38)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/3.1.4 --enable-shared --enable-pthreads --enable-gpl --enable-version3 --enable-hardcoded-tables --enable-avresample --cc=clang --host-cflags= --host-ldflags= --enable-opencl --enable-libx264 --enable-libmp3lame --enable-libxvid --disable-lzma --enable-vda
  libavutil      55. 28.100 / 55. 28.100
  libavcodec     57. 48.101 / 57. 48.101
  libavformat    57. 41.100 / 57. 41.100
  libavdevice    57.  0.101 / 57.  0.101
  libavfilter     6. 47.100 /  6. 47.100
  libavresample   3.  0.  0 /  3.  0.  0
  libswscale      4.  1.100 /  4.  1.100
  libswresample   2.  1.100 /  2.  1.100
  libpostproc    54.  0.100 / 54.  0.100
Input #0, image2, from '%04d.png':
  Duration: 00:00:02.00, start: 0.000000, bitrate: N/A
    Stream #0:0: Video: png, rgba(pc), 800x600 [SAR 3937:3937 DAR 4:3], 25 fps, 25 tbr, 25 tbn, 25 tbc
[libx264

In [64]:
# delete all png files in directory
filelist = [ f for f in os.listdir(".") if f.endswith(".png") ]

for f in filelist:
    os.remove(f)

In [17]:
# save section of video without points
# plot points on each frame of video

cap = cv2.VideoCapture(aa)
ctr = 0

# reset frame num to zero
cap.set(cv2.cv.CV_CAP_PROP_FRAME_COUNT, 0) 


while(ctr <= 600):  # check !
    # capture frame-by-frame
    ret, frame = cap.read()

    if ret and ctr > 500: # if we can return a frame, and it's over a certain frame number
        # our operation on frame come here
        #trueCol = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        # adjust gamma
        trueCol = adjust_gamma(frame, 2)
        
        # save frame
        cv2.imwrite(str(ctr).zfill(4) + '.png', trueCol)
        
    print ctr
    ctr +=1

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
# When everything is done release the capture
cap.release()
cv2.destroyAllWindows()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [18]:
stta = time.time()
!ffmpeg -start_number 500 -i %04d.png -vf "scale=trunc(iw/2)*2:trunc(ih/2)*2" -c:v libx264 -pix_fmt yuv420p -y unAnnotatedVid2.mp4
print time.time() - stta # ~ 1.4 seconds

ffmpeg version 3.1.4 Copyright (c) 2000-2016 the FFmpeg developers
  built with Apple LLVM version 8.0.0 (clang-800.0.38)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/3.1.4 --enable-shared --enable-pthreads --enable-gpl --enable-version3 --enable-hardcoded-tables --enable-avresample --cc=clang --host-cflags= --host-ldflags= --enable-opencl --enable-libx264 --enable-libmp3lame --enable-libxvid --disable-lzma --enable-vda
  libavutil      55. 28.100 / 55. 28.100
  libavcodec     57. 48.101 / 57. 48.101
  libavformat    57. 41.100 / 57. 41.100
  libavdevice    57.  0.101 / 57.  0.101
  libavfilter     6. 47.100 /  6. 47.100
  libavresample   3.  0.  0 /  3.  0.  0
  libswscale      4.  1.100 /  4.  1.100
  libswresample   2.  1.100 /  2.  1.100
  libpostproc    54.  0.100 / 54.  0.100
Input #0, image2, from '%04d.png':
  Duration: 00:00:04.00, start: 0.000000, bitrate: N/A
    Stream #0:0: Video: png, rgb24(pc), 1280x1024, 25 fps, 25 tbr, 25 tbn, 25 tbc
[libx264 @ 0x7ff604001e00] us

In [19]:
# delete all png files in directory
filelist = [ f for f in os.listdir(".") if f.endswith(".png") ]

for f in filelist:
    os.remove(f)